# 3D Plot Fractals
- This is essentially an improved and optimized version of my chaotic IFS fractal generator except for 3D
- Code by Michael Sherif Naguib 7/30/19 @Tulsa University
### Get the imports

In [98]:
#Get the imports
import tqdm
import pptk
import numpy as np
import random
import math

### Predefine colors

In [204]:
#Colors for transforms
color_a =[1,0,0]#Red
color_b= [0,1,0]#Green
color_c = [0,0,1]#Blue

### Make a system For generating IFS

In [302]:
#Calculates the Iterated Function System for a certain IFS Function, Scale and point Quantity
def IFS(func,quantity= 1000000,scale= 100000,start_point=None):
    #Store the Point Data: start with the initial start point
    pointData = [np.array([1,1,1]) if not str(type(start_point))=="<class 'numpy.ndarray'>" else start_point]#starting point
    #Build the Point Data
    for i in tqdm.tqdm(range(0,quantity)):
        #get the next point 
        pointData.append(func(pointData[i]))
    #Convert to a np Matrix and Scale
    pointData = scale*np.array(pointData)
    return pointData

#Takes a matrix of point data and plots it
def view(pointData):
    v = pptk.viewer(pointData) 
    v.set(point_size=1)

#Takes a list of transformation functions that each take a vector as an input and returns a curried function
# that will select at random one of the transformation functions when called based upon the specified weighted
#probabilities... if no weights are passed it is assumed to be uniform random
def makeIFSFunc(ruleSet,probs=False):
    if not bool(probs):#uniform random
        return lambda inputVec: ruleSet[random.randint(0,len(ruleSet)-1)](inputVec)
    else:#Else select from the weighted probabilities!
        return lambda inputVec: ruleSet[(np.random.choice(len(probs), 1, p=probs))[0]](inputVec)

### Define a LOT of constants for various fractals

In [311]:
#METHODOLOGY
# use this area to predefine constants
# build a list of functions that preform the desired vector operations on the input vector
# (optional) build another list that holds the probabilities for that transformation
# finally  in the dictionary convert the list and or probs to a single IFS function.... 

#For Convenience of Coding
m = np.matmul
I = np.identity(3)
s=sin= math.sin
c=cos= math.cos
sqrt = math.sqrt
pi = math.pi
rr = random.random
na = np.array
def Rx(t):# Euler Angle Rotation Matricies
    return np.array([[1,0,0],[0,c(t),0-s(t)],[0,s(t),c(t)]])
def Ry(t):
    return np.array([[c(t),0,s(t)],[0,1,0],[0-s(t),0,c(t)]])
def Rz(t):
    return np.array([[c(t),0-s(t),0],[s(t),c(t),0],[0,0,1]])
    
#Define a ruleset for Hedgehog fractal
hedgeHog=[]#Credit for the values of the constants goes to Paul Bourke http://paulbourke.net/fractals/ifs/
a = (1 + 1/3)/2
b = (1 - 1/3)/2
hedgeHog.append(lambda v: m((1/3)*I,v))#Oddly this resembles a binary bit string (which values of a are negative)
hedgeHog.append(lambda v: m((b*I),v ) + np.array([a,a,a]))
hedgeHog.append(lambda v: m((b*I),v ) + np.array([a,a,0-a]))
hedgeHog.append(lambda v: m((b*I),v ) + np.array([a,0-a,a]))
hedgeHog.append(lambda v: m((b*I),v ) + np.array([a,0-a,0-a]))
hedgeHog.append(lambda v: m((b*I),v ) + np.array([0-a,a,a]))
hedgeHog.append(lambda v: m((b*I),v ) + np.array([0-a,a,0-a]))
hedgeHog.append(lambda v: m((b*I),v ) + np.array([0-a,0-a,a]))
hedgeHog.append(lambda v: m((b*I),v ) + np.array([0-a,0-a,0-a]))

#Define a ruleset for Serinpenski fractal
serinpenski = []
serinpenski.append(lambda v: m((1/2)*I,v)+np.array([0,0,0.6]))
serinpenski.append(lambda v: m((1/2)*I,v)+np.array([0,0.6,0]))
serinpenski.append(lambda v: m((1/2)*I,v)+np.array([0.6,0,0]))
serinpenski.append(lambda v: m((1/2)*I,v))

#Define a ruleset for a Dragon fractal
hyper_constant = 1.8
d = sqrt(3)/3
e = hyper_constant*sqrt(2)/4
rot_a = m(Ry(pi/6),Rx(pi/6))
rot_b = m(Ry(pi/3),Rx(pi/3))
dragon=[]
dragon.append(lambda v:m(rot_a,(e*v)))
dragon.append(lambda v:m(rot_b,(e*v)) + np.array([1,0,d]))
dragon.append(lambda v:m(rot_b,(e*v)) + np.array([d,0,1]))

#Define a ruleset for a ORB Like fractal: Best result use 10^5 points or less
orb= []
orb.append(lambda v: m(Rx(pi/6),v) + np.array([-v[i]*20/abs(v[i]) for i in range(0,3)]))
orb.append(lambda v: m(Ry(pi/180),v))

#Define a ruleset for a Serinpenski Variant Like fractal: Best result use 10^5 points or less
serinpenski2=[]
serinpenski2.append(lambda v: m(0.5*I,v))
serinpenski2.append(lambda v: m(0.5*I,v) + na([0,0,0.5]))
serinpenski2.append(lambda v: m(0.5*I,v) + na([0.5,0,0.5]))
serinpenski2.append(lambda v: m(0.5*I,v)+ na([0,0,0.5]))
serinpenski2.append(lambda v: m(0.5*I,v)+ na([0.25,0.5,0.25]))

### Create a list of IFS Functions

In [312]:
#Create the chaotic version of the function given a list of transform functions (random uniform)
allIFS = {
    "hedgehog": makeIFSFunc(hedgeHog),
    "triangle": makeIFSFunc(serinpenski),
    "dragon":   makeIFSFunc(dragon),
    "orb":      makeIFSFunc(orb),
    "triangle2":makeIFSFunc(serinpenski2)
}

### Run a calculation

In [315]:
#Calculate the data based off the function ... (uses default scaling and quantity)
data = IFS(allIFS["s2"],quantity=int(math.pow(10,5)),start_point=np.array([1,1,0]))

100%|███████████████████████████████| 100000/100000 [00:01<00:00, 98011.11it/s]


### View the Data

In [316]:
view(data)
# Axis Colors & Index: 
#Green: x    0
#Blue:  y    1
#Red:   z    2 